In [1]:
using LinearAlgebra
using Plots

In [2]:
fi = ComplexF32(1.0im)

0.0f0 + 1.0f0im

In [3]:
function vec2hermite(v::Vector{Float32})
    N = round(Int, sqrt(length(v)))
    H = zeros(ComplexF32, N, N)
    for i in 1:N
        for j in i:N
            l = N*(i-1) + 2j - i^2
            if(i == j)
                H[i,j] = v[l]
            else
                H[i,j] = v[l-1] + fi*v[l]
            end 
        end
    end
    H = Hermitian(H)
    return H
end

vec2hermite (generic function with 1 method)

In [4]:
function vec2unitary(v::Vector{Float32}, τ::Float32)
    H = vec2hermite(v)
    U = exp(fi*(τ*H))
    return U
end

vec2unitary (generic function with 1 method)

In [5]:
function make_unitary(N::Int, τ::Float32)
    v = rand(Float32, N^2)
    U = vec2unitary(v, τ)
    return U
end

make_unitary (generic function with 1 method)

In [6]:
test = rand(Float32, 9)

9-element Vector{Float32}:
 0.356286
 0.31322783
 0.74740523
 0.44164985
 0.41414195
 0.28483528
 0.4944756
 0.55824727
 0.34678203

In [7]:
H_test = vec2hermite(test)
U_test = vec2unitary(test, 0.1f0)

3×3 Matrix{ComplexF32}:
  0.994263+0.0353343im  -0.0779995+0.0269764im  -0.0426874+0.0394755im
 0.0708991+0.0351993im    0.992112+0.0742939im  -0.0467876+0.041715im
 0.0395351+0.0486157im   0.0357417+0.0458886im    0.993558+0.0743832im

In [8]:
U_test*U_test'

3×3 Matrix{ComplexF32}:
        1.0+0.0im         1.86265f-9+7.567f-9im         0.0+3.72529f-9im
 1.86265f-9-7.567f-9im           1.0+0.0im       3.72529f-9+0.0im
        0.0-3.72529f-9im  3.72529f-9+0.0im              1.0+0.0im

In [49]:
s_dim = 2
e_dim = 4
tot_dim = s_dim * e_dim
M_size = tot_dim^2

64

In [41]:
function norm!(m::Hermitian{ComplexF32, Matrix{ComplexF32}})
    T = real(tr(m))
    m = m./T
end

norm! (generic function with 1 method)

In [42]:
function make_rand_dm(dim::Int)
    ρ_vec = rand(Float32, dim^2)
    rt_ρ = vec2hermite(ρ_vec)
    ρ = Hermitian(norm!(Hermitian(rt_ρ*rt_ρ')))
    return ρ
end

make_rand_dm (generic function with 1 method)

In [12]:
function make_rand_dm_ev(dim::Int)
    ρ_vec = rand(Float32, dim^2)
    rt_ρ = vec2hermite(ρ_vec)
    e, ev = eigen(rt_ρ)
    ρ = Hermitian(norm!(rt_ρ*rt_ρ'))
    return ρ, ev
end

make_rand_dm_ev (generic function with 1 method)

In [43]:
s_dm = make_rand_dm(s_dim)
println(s_dm)
tr(s_dm)

ComplexF32[0.5512375f0 + 0.0f0im 0.49109903f0 + 0.06636295f0im; 0.49109903f0 - 0.06636295f0im 0.44876245f0 + 0.0f0im]


1.0f0

In [44]:
es, evs = eigen(s_dm)
testm = evs * Diagonal(es) * evs'
tr(testm)

0.99999994f0 + 7.275958f-12im

In [45]:
s_dm * evs[:,1] - es[1] * evs[:,1]

2-element Vector{ComplexF32}:
  4.458707f-8 + 3.3469405f-9im
 2.8172508f-8 - 1.953966f-9im

In [46]:
evs[:,1]'*evs[:,1]

0.99999994f0 - 1.6865203f-9im

In [47]:
e_dm = make_rand_dm(e_dim)
println(e_dm)
tr(e_dm)

ComplexF32[0.23526613f0 + 0.0f0im 0.19455637f0 + 0.013052387f0im 0.105433926f0 + 0.12664899f0im 0.05172421f0 + 0.17908855f0im; 0.19455637f0 - 0.013052387f0im 0.3046407f0 + 0.0f0im 0.1971553f0 + 0.10975214f0im 0.12364557f0 + 0.16229418f0im; 0.105433926f0 - 0.12664899f0im 0.1971553f0 - 0.10975214f0im 0.21607307f0 + 0.0f0im 0.20999402f0 + 0.057906352f0im; 0.05172421f0 - 0.17908855f0im 0.12364557f0 - 0.16229418f0im 0.20999402f0 - 0.057906352f0im 0.24402006f0 + 0.0f0im]


0.99999994f0

In [50]:
test_U = make_unitary(tot_dim, 1.0f0)

8×8 Matrix{ComplexF32}:
   0.303165+0.157195im     -0.33742-0.0812193im  …   0.238759+0.657219im
   0.258717+0.0416078im    0.345462-0.235346im      0.0183932-0.0697175im
   0.255102+0.00792686im   0.245624+0.0102503im     0.0508646+0.203401im
  -0.261306-0.211998im     0.137908-0.29378im        0.249837+0.243911im
   0.386219-0.203184im     0.495788-0.0279889im     0.0243188+0.142586im
  -0.240705-0.254318im     0.226626+0.0533096im  …  -0.352388-0.0839863im
 -0.0493961-0.311213im    -0.260997+0.252405im      -0.204033+0.255039im
   0.158265+0.446032im      0.11824-0.304069im       0.130095-0.253449im

In [52]:
II = test_U * test_U'
println([real(II[i,i]) for i in 1:tot_dim])

Float32[0.9999997, 0.9999996, 1.0000002, 0.99999994, 1.0, 0.99999994, 1.0000001, 1.0]


In [65]:
function twohot(i::Int, j::Int, s_dim::Int, e_dim::Int)
    v = zeros(Float32, s_dim*e_dim)
    v[i+s_dim*(j-1)] = 1.0f0
    return v
end

twohot (generic function with 2 methods)

In [54]:
function ehot(vs::Vector{ComplexF32}, i::Int, e_dim::Int)
    s_dim = length(vs)
    ve = zeros(ComplexF32, e_dim*s_dim)
    ve[(s_dim*(i-1)+1):(s_dim*i)] = vs
    #v2 = vcat(vs, ve)
    return ve
end

ehot (generic function with 1 method)

In [63]:
function make_ev(s_ev::Matrix{ComplexF32}, e_dim::Int)
    e_vec::Vector{Matrix{Float32}} = []
    s_vec::Vector{Matrix{ComplexF32}} = []
    s_dim = size(s_ev)[1]
    tot_dim = s_dim * e_dim
    println(s_dim)
    for i in 1:s_dim
        sm = zeros(ComplexF32, tot_dim, e_dim)
        for j in 1:e_dim
            sm[:,j] = ehot(s_ev[:,i], j, e_dim)
            #push!(e_vec, twohot(i, j, s_dim))
            #push!(s_vec, ehot(s_ev[:,i], j, e_dim))
        end
        push!(s_vec, sm)
    end

    for j in 1:e_dim
        em = zeros(Float32, tot_dim, s_dim)
        for i in 1:s_dim
            em[:,i] = twohot(i, j, s_dim, e_dim)
        end
        push!(e_vec, em)
    end
    return s_vec, e_vec
end

make_ev (generic function with 1 method)

In [66]:
s_evs, e_evs = make_ev(evs, e_dim)

2


(Matrix{ComplexF32}[[0.6637269f0 + 0.08969041f0im 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im; -0.742578f0 + 0.0f0im 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im; … ; 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im 0.6637269f0 + 0.08969041f0im; 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im -0.742578f0 + 0.0f0im], [0.73588955f0 + 0.099441856f0im 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im; 0.6697595f0 + 0.0f0im 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im; … ; 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im 0.73588955f0 + 0.099441856f0im; 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im 0.6697595f0 + 0.0f0im]], Matrix{Float32}[[1.0 0.0; 0.0 1.0; … ; 0.0 0.0; 0.0 0.0], [0.0 0.0; 0.0 0.0; … ; 0.0 0.0; 0.0 0.0], [0.0 0.0; 0.0 0.0; … ; 0.0 0.0; 0.0 0.0], [0.0 0.0; 0.0 0.0; … ; 1.0 0.0; 0.0 1.0]])

In [68]:
e_evs[1]'Diagonal(ones(8))*e_evs[1]

2×2 Matrix{Float64}:
 1.0  0.0
 0.0  1.0

In [70]:
function make_Mk(U::Matrix{ComplexF32}, s_vec::Vector{Matrix{ComplexF32}}, e_vec::Vector{Matrix{Float32}})
    L = size(U)[1]
    e_dim = length(s_vec)
    s_dim = div(L,e_dim)
    Ms::Vector{Matrix{ComplexF32}} = []
    for j in 1:e_dim
        for k in 1:s_dim
            push!(Ms, (e_vec[k]'*U*s_vec[j]))
        end
    end
    return Ms
end

make_Mk (generic function with 1 method)

In [71]:
Ms = make_Mk(test_U, s_evs, e_evs)

8-element Vector{Matrix{ComplexF32}}:
 [0.43768018f0 + 0.19183713f0im -0.31465524f0 - 0.14959966f0im -0.079657026f0 - 0.19463558f0im -0.31955475f0 - 0.41063914f0im; -0.08854665f0 + 0.2255836f0im -0.12683375f0 + 0.22426392f0im -0.020753548f0 - 0.29442832f0im 0.26760727f0 + 0.020996764f0im]
 [-0.013788182f0 + 0.020529808f0im 0.6399907f0 - 0.15555431f0im -0.3944964f0 - 0.17374477f0im 0.10507965f0 + 0.0084815705f0im; -0.25682884f0 + 0.054009352f0im 0.122937895f0 + 0.38588935f0im 0.17827128f0 + 0.2736153f0im -0.26790026f0 - 0.23143224f0im]
 [-0.09359364f0 - 0.079434715f0im 0.0770683f0 - 0.07548167f0im 0.47533765f0 + 0.13831753f0im -0.21186672f0 - 0.108434f0im; -0.3052402f0 - 0.22997293f0im -0.33100596f0 - 0.057116725f0im -0.29592726f0 - 0.18400033f0im 0.14939609f0 + 0.23932214f0im]
 [0.18893825f0 - 0.3984214f0im 0.018131372f0 - 0.040610347f0im 0.4360096f0 - 0.051609118f0im 0.5718192f0 - 0.18190897f0im; -0.022762207f0 + 0.5360336f0im -0.29577455f0 - 0.022286532f0im 0.12835771f0 - 0.021023622

In [72]:
test = sum([Ms[i]'*es[div(i-1,e_dim)+1]*Ms[i] for i in 1:e_dim*s_dim])

4×4 Matrix{ComplexF32}:
         1.0+6.14039f-9im  …  -6.70552f-8+1.49012f-8im
  1.06171f-7+6.42613f-8im     -1.49012f-8-7.82311f-8im
  1.93715f-7+5.96046f-8im       8.9407f-8-4.47035f-8im
 -6.70552f-8+0.0im                    1.0+2.60079f-10im

In [78]:
test2 = sum([Ms[i]'*es[((i-1)%s_dim)+1]*Ms[i] for i in 1:e_dim*s_dim])

4×4 Matrix{ComplexF32}:
   0.906017+8.23896f-10im  -0.0435312+0.209154im    …  0.280127+0.265974im
 -0.0435312-0.209154im        1.18342-1.70095f-9im     0.173691+0.11145im
  0.0175821+0.24567im       -0.174167+0.358149im       0.235622+0.13258im
   0.280127-0.265974im       0.173691-0.11145im        0.935342+1.34467f-9im

In [75]:
test3 = sum([Ms[i]*es[div(i-1,e_dim)+1]*Ms[i]' for i in 1:e_dim*s_dim])

2×2 Matrix{ComplexF32}:
  1.62761-4.16471f-9im  0.498016+0.376122im
 0.498016-0.376122im     2.37238+3.69774f-8im

In [77]:
test4 = sum([Ms[i]*es[((i-1)%s_dim)+1]*Ms[i]' for i in 1:e_dim*s_dim])

2×2 Matrix{ComplexF32}:
        2.0-1.08904f-9im  7.45058f-8-1.19209f-7im
 7.45058f-8+1.3411f-7im          2.0+1.72627f-8im